In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader
from torchviz import make_dot
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

from sklearn import svm
from sklearn.model_selection import train_test_split

import time
import sys
import wandb
from tqdm.notebook import tqdm

sys.path.append("/home/cygnus/fujimoto/Cygnus-X_CAE/Cygnus-X_cloud")
from models.Binary_classification import Binary_classification as model

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
class DataSet():
    def __init__(self, data, label):
        self.label = label
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.label[index]

In [3]:
latent = 100
model = model.Binary_classification(latent)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [4]:
model.to(device)
model.eval()

Binary_classification(
  (encoder): Sequential(
    (0): Conv3d(1, 32, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(32, 32, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (7): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (10): ReLU(inplace=True)
    (11): Conv3d(64, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (12): ReLU(inplace=True)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=84640, out_features=100, bias=True)
    (15): Linear(in_features=100, 

In [5]:
summary(model.to(device), (1, 30, 100, 100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 15, 50, 50]           2,080
       BatchNorm3d-2       [-1, 32, 15, 50, 50]              64
              ReLU-3       [-1, 32, 15, 50, 50]               0
            Conv3d-4        [-1, 32, 7, 25, 25]          65,568
       BatchNorm3d-5        [-1, 32, 7, 25, 25]              64
              ReLU-6        [-1, 32, 7, 25, 25]               0
            Conv3d-7        [-1, 64, 7, 25, 25]          55,360
       BatchNorm3d-8        [-1, 64, 7, 25, 25]             128
              ReLU-9        [-1, 64, 7, 25, 25]               0
           Conv3d-10        [-1, 64, 7, 25, 25]         110,656
             ReLU-11        [-1, 64, 7, 25, 25]               0
           Conv3d-12        [-1, 32, 5, 23, 23]          55,328
             ReLU-13        [-1, 32, 5, 23, 23]               0
          Flatten-14                [-1

In [6]:
#dataには学習データのpathを入力する
data = np.load("/home/cygnus/fujimoto/Cygnus-X_CAE/data/zroing_resize_data/resize_data/condition_match/vflip/CygnusX_cut_truncation_3size_resize_to_mode_percentile_100x100.npy")
print(len(data))
data = torch.from_numpy(data).float()
label = [0]*len(data)

train_data, val_data, train_labels, val_labels = train_test_split(data, label, test_size=0.2, random_state=42, stratify=label)
print(type(train_data))
val_data, test_data, val_labels, test_labels = train_test_split(val_data, val_labels, test_size=0.25, random_state=42, stratify=val_labels)

train_dataset = DataSet(train_data, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

val_dataset = DataSet(val_data, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

test_dataset = DataSet(test_data, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

7925
<class 'torch.Tensor'>


In [7]:
model.to(device)

output_list = []
for images, labels in tqdm(test_dataloader):
    images = images.view(-1, 1, 30, 100, 100)  # バッチサイズを維持したままチャンネル数を1に設定
    with torch.set_grad_enabled(True):
            
        # モデルの出力を計算する
        output = model(images.clone().to(device))
        output = output.squeeze()
        output_list.append(output.to("cpu").detach().numpy().copy())

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
print(output_list)

[array([0.48781744, 0.4878497 , 0.48780087, 0.48787594, 0.4878147 ,
       0.4877868 , 0.4877987 , 0.48784545], dtype=float32), array([0.4878632 , 0.48778582, 0.4878307 , 0.487906  , 0.4879473 ,
       0.48789257, 0.48786372, 0.4879466 ], dtype=float32), array([0.4878322 , 0.4877506 , 0.4878378 , 0.4877931 , 0.48779598,
       0.48785543, 0.4878962 , 0.48787144], dtype=float32), array([0.4878887 , 0.48795986, 0.48785362, 0.48781878, 0.48777926,
       0.48765913, 0.4878165 , 0.48786044], dtype=float32), array([0.48783422, 0.48778033, 0.48781392, 0.4879207 , 0.48776627,
       0.4877431 , 0.48785943, 0.487733  ], dtype=float32), array([0.48787972, 0.4878583 , 0.4878619 , 0.48784694, 0.4878182 ,
       0.4878028 , 0.4878635 , 0.48785636], dtype=float32), array([0.4878037 , 0.487872  , 0.48786485, 0.48785073, 0.4878701 ,
       0.48778695, 0.48784444, 0.48784012], dtype=float32), array([0.48786765, 0.4879039 , 0.48787972, 0.48775584, 0.4879079 ,
       0.48785236, 0.48786542, 0.48787677],